In [1]:
import netaddr, time, sys, json, zipfile, csv
import pandas as pd
from zipfile import ZipFile
from io import TextIOWrapper

In [2]:
df = pd.read_csv("ip2location.csv")

In [3]:
ips = ["1.1.1.1","1.1.1.2","9.9.9.9","1.1.1.2"]

In [4]:
def dict_creator(ip, index, t0):
    info = {}
    info["ip"] = ip 
    info["int_ip"] = int(netaddr.IPAddress(ip)) 
    info["region"] = df["region"][index-1] 
    info["ms"] = (time.time() - t0) * 1000
    return info

def ip_test(ips, index=0, result=[]):
    while len(ips) > 0:
        info = {}
        ip = ips.pop(0)
        index %= (len(df) - 1)
        t0 = time.time()
        if df["low"].at[index+1] <= int(netaddr.IPAddress(ip)) <= df["high"].at[index+1]:
            result.append(dict_creator(ip, index+1, t0))
            break
        else:
            for low in df["low"][index:]:
                if low <= int(netaddr.IPAddress(ip)) <= df["high"].at[index]:
                    result.append(dict_creator(ip, index+1, t0))
                    break
                else:
                    index += 1
        if index == 174972:
            index = 0
            for low in df["low"][index:]:
                if low <= int(netaddr.IPAddress(ip)) <= df["high"].at[index]:
                    result.append(dict_creator(ip, index+1, t0))
                    break
                else:
                    index += 1
    return result

In [5]:
dict_creator("1.1.1.1", 11, .5)

{'ip': '1.1.1.1',
 'int_ip': 16843009,
 'region': 'United States of America',
 'ms': 1617680761660.4973}

In [6]:
ips = ["1.1.1.1","1.1.1.2","9.9.9.9","1.1.1.2"]
ips2 = ["1.1.1.1", "5.5.5.5", "9.9.9.9", "7.7.7.7", "5.5.5.4", "5.5.5.5", "5.5.5.6", "5.5.5.7", "5.5.5.8", "5.5.5.9"]
single_ip = ["5.5.5.5"]
ip_test(ips2, 0, [])

[{'ip': '1.1.1.1',
  'int_ip': 16843009,
  'region': 'United States of America',
  'ms': 2.0287036895751953},
 {'ip': '5.5.5.5',
  'int_ip': 84215045,
  'region': 'Germany',
  'ms': 15.099763870239258},
 {'ip': '9.9.9.9',
  'int_ip': 151587081,
  'region': 'United States of America',
  'ms': 21.68130874633789},
 {'ip': '7.7.7.7',
  'int_ip': 117901063,
  'region': 'United States of America',
  'ms': 466.1517143249512},
 {'ip': '5.5.5.4',
  'int_ip': 84215044,
  'region': 'Germany',
  'ms': 450.7565498352051},
 {'ip': '5.5.5.5',
  'int_ip': 84215045,
  'region': 'Germany',
  'ms': 0.1087188720703125},
 {'ip': '5.5.5.6',
  'int_ip': 84215046,
  'region': 'Germany',
  'ms': 0.0858306884765625},
 {'ip': '5.5.5.7',
  'int_ip': 84215047,
  'region': 'Germany',
  'ms': 0.08034706115722656},
 {'ip': '5.5.5.8',
  'int_ip': 84215048,
  'region': 'Germany',
  'ms': 0.07653236389160156},
 {'ip': '5.5.5.9',
  'int_ip': 84215049,
  'region': 'Germany',
  'ms': 0.08487701416015625}]

In [7]:
df["region"][1098]

'Germany'

In [8]:
test = len(df) + 1
test %= len(df)
test

1

In [9]:
df["low"].at[5]

16781312

In [ ]:
zip1 = "large.zip"
zip2 = "small.zip"

def zip_csv_iter(name):
    with ZipFile(name) as zf:
        with zf.open(name.replace(".zip", ".csv")) as f:
            reader = csv.reader(TextIOWrapper(f))
            print(reader[0])
            for row in reader:
                yield row

def sorting(instance):
    first_digits = int(instance[0].split(".")[0])
    second_digits = int(instance[0].split(".")[1])
    third_digits = int(instance[0].split(".")[2])
    return (first_digits, second_digits, third_digits)

def sampling(zip1, zip2, stride=30000):    
    reader = zip_csv_iter(zip1)
    header = next(reader) # the list of all column names
    header.append("region")
    ip_idx = header.index("ip")
    samples = []
    while True:
        row = next(reader)
        ip_str = ""
        for character in row[ip_idx]:
            if character.isalpha():
                ip_str += "0"
            else:
                ip_str += character 
        row.append(ip_test([ip_str])[-1]["region"])
        samples.append(row)
        for i in range(int(stride)-1):
            try:
                next(reader)
            except StopIteration:
                samples.sort(key=sorting)
                print(samples)
                with ZipFile(zip2, "w") as zf:
                    with zf.open(zip2.replace(".zip", ".csv"), "w") as raw:
                        with TextIOWrapper(raw) as f:
                            writer = csv.writer(f, lineterminator='\n')
                            writer.writerow(header) # write the row of column names to zip2
                            for row in samples:
                                writer.writerow(row) # write a row to zip2
                return

In [ ]:
sampling("large.zip", "small_samp.zip", 30000)

In [12]:
def sorting(instance):
    first_digits = int(instance[0].split(".")[0])
    second_digits = int(instance[0].split(".")[1])
    third_digits = int(instance[0].split(".")[2])
    return (first_digits, second_digits, third_digits)

def sampling(zip1, zip2, stride=30000):    
    reader = zip_csv_iter(zip1)
    header = next(reader) # the list of all column names
    header.append("region")
    ip_idx = header.index("ip")
    samples = []
    while True:
        row = next(reader)
        ip_str = ""
        for character in row[ip_idx]:
            if character.isalpha():
                ip_str += "0"
            else:
                ip_str += character 
        row.append(ip_test([ip_str])[-1]["region"])
        samples.append(row)
        for i in range(int(stride)-1):
            try:
                next(reader)
            except StopIteration:
                samples.sort(key=sorting)
                with ZipFile(zip2, "w") as zf:
                    with zf.open(zip2.replace(".zip", ".csv"), "w") as raw:
                        with TextIOWrapper(raw) as f:
                            writer = csv.writer(f, lineterminator='\n')
                            writer.writerow(header) # write the row of column names to zip2
                            for row in samples:
                                writer.writerow(row) # write a row to zip2
                return

In [13]:
zip1 = "large.zip"
zip2 = "small.zip"

def sorting(instance):
    first_digits = int(instance.split(".")[0])
    second_digits = int(instance.split(".")[1])
    third_digits = int(instance.split(".")[2])
    return (first_digits, second_digits, third_digits)

def sampling(zip1, zip2, stride=30000):
    reader = zip_csv_iter(zip1)
    header = next(reader)
    header.append("region")
    ip_idx = header.index("ip")
    samples = []
    ips = []
    while True:
        row = next(reader)
        #samples.append(row)
        ip_str = ""
        for character in row[ip_idx]:
            if character.isalpha():
                ip_str += "0"
            else:
                ip_str += character
        ips.append(ip_str)
        for i in range(int(stride)-1):
            try:
                next(reader)
            except StopIteration:
                ips.sort(key=sorting)    
                return ip_test(ips) 
            
            

In [14]:
sorting("104.197.32.000")

(104, 197, 32)

In [15]:
def dict_creator(ip, index, t0):
    info = {}
    info["ip"] = ip 
    info["int_ip"] = int(netaddr.IPAddress(ip)) 
    info["region"] = df["region"][index] 
    info["ms"] = (time.time() - t0) * 1000
    return info

def ip_test(ips, index=0, result=list()):
    last_index = 0
    while len(ips) > 0:
        info = {}
        ip = ips.pop(0)
        index %= (len(df) - 1)
        t0 = time.time()
        print(last_index)
        if df["low"].at[last_index] <= int(netaddr.IPAddress(ip)) <= df["high"].at[last_index]:
            result.append(dict_creator(ip, last_index, t0))
            print("recursive", last_index)
        else:
            for low in df["low"][index:]:
                if low <= int(netaddr.IPAddress(ip)) <= df["high"].at[index]:
                    result.append(dict_creator(ip, index, t0))
                    last_index = index
                    print("linear", index)
                else:
                    index += 1
    return result

In [16]:
#ips = ["1.1.1.1", "1.1.1.2", "9.9.9.9", "1.1.1.2"]
ips2 = ["1.1.1.1", "5.5.5.5", "9.9.9.9", "7.7.7.7", "5.5.5.4", "5.5.5.5", "5.5.5.6", "5.5.5.7", "5.5.5.8", "5.5.5.9"]
#ip_test(ips, 0, [])
#for ip in sampling(zip1, zip2, 20000):
    
ips = sampling(zip1, zip2, 1000)
ips


0
linear 7434
7434
recursive 7434
7434
linear 13397
13397
linear 16341
16341
recursive 16341
16341
linear 27344
27344
linear 28590
28590
recursive 28590
28590
linear 28591
28591
recursive 28591
28591
linear 28592
28592
linear 28594
28594
linear 28595
28595
recursive 28595
28595
recursive 28595
28595
recursive 28595
28595
recursive 28595
28595
recursive 28595
28595
recursive 28595
28595
recursive 28595
28595
linear 28653
28653
linear 28745
28745
recursive 28745
28745
linear 29950
29950
linear 29951
29951
recursive 29951
29951
recursive 29951
29951
recursive 29951
29951
recursive 29951
29951
linear 29959
29959
linear 29963
29963
recursive 29963
29963
recursive 29963
29963
recursive 29963
29963
recursive 29963
29963
recursive 29963
29963
recursive 29963
29963
recursive 29963
29963
recursive 29963
29963
recursive 29963
29963
recursive 29963
29963
recursive 29963
29963
recursive 29963
29963
recursive 29963
29963
recursive 29963
29963
recursive 29963
29963
recursive 29963
29963
linear 30063


KeyboardInterrupt: 

In [ ]:
def dict_creator(ip, index, t0):
    info = {}
    info["ip"] = ip 
    info["int_ip"] = int(netaddr.IPAddress(ip)) 
    info["region"] = df["region"][index] 
    info["ms"] = (time.time() - t0) * 1000
    return info

def ip_test(ips, index=0, result=list()):
    last_index = 0
    while len(ips) > 0:
        index = 0
        ip = ips.pop(0)
        index %= (len(df) - 1)
        last_index %= (len(df) - 1)
        t0 = time.time()
        #search starting at index of last ip
        for low in df["low"][last_index:]:
            if low <= int(netaddr.IPAddress(ip)) <= df["high"].at[last_index + index]:
                result.append(dict_creator(ip, last_index + index, t0))
                print(last_index, index)
                last_index += index
                print("recursive")
            else:
                index += 1
        if index == len(df):
            index = 0
            for low in df["low"][index:]:
                if low <= int(netaddr.IPAddress(ip)) <= df["high"].at[index]:
                    result.append(dict_creator(ip, index, t0))
                    print("linear")
                else:
                    index += 1
    return result

In [ ]:
ips = ["1.1.1.1", "1.1.1.2", "9.9.9.9", "1.1.1.2"]
ip_test(ips)

In [ ]:
def dict_creator(ip, index, t0):
    info = {}
    info["ip"] = ip 
    info["int_ip"] = int(netaddr.IPAddress(ip)) 
    info["region"] = df["region"][index] 
    info["ms"] = (time.time() - t0) * 1000
    return info

def ip_test(ips, index=0, result=list()):
    last_index = 0
    while len(ips) > 0:
        info = {}
        ip = ips.pop(0)
        index %= (len(df) - 1)
        t0 = time.time()
        print(ip, last_index, index)
        if df["low"].at[last_index] <= int(netaddr.IPAddress(ip)) <= df["high"].at[last_index]:
            result.append(dict_creator(ip, last_index, t0))
            print("recursive", last_index)
        else:
            for low in df["low"][last_index:]:
                if low <= int(netaddr.IPAddress(ip)) <= df["high"].at[last_index + index]:
                    result.append(dict_creator(ip, last_index + index, t0))
                    last_index = index
                    print("linear", index)
                else:
                    index += 1
    return result

In [ ]:
ips = ["1.1.1.1", "1.1.1.2", "9.9.9.9", "1.1.1.2"]
ip_test(ips)

In [19]:
with ZipFile("small.zip") as zf:
        with zf.open("small.zip".replace(".zip", ".csv")) as f:
            reader = csv.reader(TextIOWrapper(f))
            print(reader[0])

TypeError: '_csv.reader' object is not subscriptable